In [91]:
%run ../pythonfile/imports
pyautogui.hotkey("Shift", "Enter")

In [92]:
from pyforest import *
import fire
import pythran
import pyautogui
import cython
from typing import List
pyautogui.hotkey("Shift", "Enter")

In [93]:
p = Path("../datasets/Excel2013_sample")
paths:List = list(p.glob("**/*.xlsx"))
xl1 = pd.read_excel(paths[0])
pyautogui.hotkey("Shift", "Enter")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [94]:
xl1

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,2013-01-25 00:00:00
0,第4四半期東地区売上,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,(単位：千円)
2,NaN,札幌,仙台,東京,神奈川,合計
3,10月,1420,2210,3220,2650,9500
4,11月,1540,2180,3120,2460,9300
5,12月,1490,1910,2890,2450,8740
6,売上実績,4450,6300,9230,7560,27540
7,月平均,1483.33,2100,3076.67,2520,9180
8,売上目標,4500,6200,9200,7500,27400
9,差額,-50,100,30,60,140


### 行列・ベクトルの操作

#### - transpose：転置

##### [転置行列の性質](https://mathtrain.jp/transpose)

・（転置行列のトレース，行列式）任意の正方行列 A に対して，

1． $\operatorname { tr } A ^ { \top } = \operatorname { tr } A$

2． $\operatorname { det } A ^ { \top } = \operatorname { det } A$

・（転置行列の積）積 AB が定義できるサイズのとき

3．$( A B ) ^ { \top } = B ^ { \top } A ^ { \top }$

・（転置行列の逆行列）A が正則なとき，

4．$\left( A ^ { \top } \right) ^ { - 1 } = \left( A ^ { - 1 } \right) ^ { \top }$

In [48]:
arr1 = xl1["Unnamed: 1"]
arr1.shape

(11,)

#### - インデックスによる要素アクセス（Ingteger array indexing）

インデックスを示す配列を与えてやることで、配列の一部を抽出することができます。

参考：(Advanced Indexing) http://docs.scipy.org/doc/numpy-1.10.0/reference/arrays.indexing.html

In [49]:
arr1[[0, 7, 0, 1]]

0          0
7    1483.33
0          0
1          0
Name: Unnamed: 1, dtype: object

In [50]:
arr1 * 2

0           0
1           0
2           0
3        2840
4        3080
5        2980
6        8900
7     2966.67
8        9000
9        -100
10    1.97778
Name: Unnamed: 1, dtype: object

#### - 集約演算 (sum, mean, max, minなど)

集約演算では、何も指定しなければ対象全体に対して演算を考えます。

引数として `axis` を指定すれば、その軸に沿って演算を行うことができます。

例えば、`x.shape=(2,3,4)` について考えると、

- `axis=0`：`x[0,:,:]`と`x[1,:,:]`の(3,4)x2について要素ごとに演算

- `axis=1`：`x[:,0,:]`と`x[:,1,:]`と`x[:,2,:]`の(2,4)x3について要素ごとに演算

- `axis=2`：`x[:,:,0]`と`x[:,:,1]`と`x[:,:,2]`と`x[:,:,3]`の(2,3)x4について要素ごとに演算

In [82]:
@cython.cfunc
def TypeInt(array):
    sums:int = 0
    counter:int = 0
    for i in array:
        try:
            sums += int(i)
        except:
            array[counter] = 0
            pass
        counter += 1
    return array, sums


converted1 ,sum1 = TypeInt(arr1)
print(converted1,"\n", sum1)

0            0
1            0
2            0
3         1420
4         1540
5         1490
6         4450
7      1483.33
8         4500
9          -50
10    0.988889
Name: Unnamed: 1, dtype: object 
 14833


In [78]:
arr1.mean(), arr1.max(), arr1.min()

(1348.5747474747475, 4500, -50)

In [36]:
# Squared norm
np.linalg.norm(arr1, ord=2)

<IPython.core.display.Javascript object>

6990.234527945326

In [47]:
arr2 = xl1[xl1.columns[-1]]
arr2

0         NaN
1     (単位：千円)
2          合計
3        9500
4        9300
5        8740
6       27540
7        9180
8       27400
9         140
10    1.00511
Name: 2013-01-25 00:00:00, dtype: object

In [83]:
converted2, sum2 = TypeInt(arr2)

print(converted2,"\n", sum2)

0           0
1           0
2           0
3        9500
4        9300
5        8740
6       27540
7        9180
8       27400
9         140
10    1.00511
Name: 2013-01-25 00:00:00, dtype: object 
 91801


In [84]:
# Concatenating arrays
mat = np.hstack((converted1, converted2))
mat

<IPython.core.display.Javascript object>

array([0, 0, 0, 1420, 1540, 1490, 4450, 1483.3333333333333, 4500, -50,
       0.9888888888888889, 0, 0, 0, 9500, 9300, 8740, 27540, 9180, 27400,
       140, 1.005109489051095], dtype=object)

In [68]:
mat2 = mat.reshape(2, 11)
mat2

array([[0, 0, 0, 1420, 1540, 1490, 4450, 1483.3333333333333, 4500, -50,
        0.9888888888888889],
       [0, 0, 0, 9500, 9300, 8740, 27540, 9180, 27400, 140,
        1.005109489051095]], dtype=object)

#### - 行列積・テンソル積（matmul (dot), tensordot, einsum）
- matmul  
  行列積や行列とベクトルの積にはmatmulを使用するのが最も簡単です。  
  $$C = AB \Leftrightarrow c_{ik} = \sum_{j} a_{ij}b_{jk}$$
  3階以上のテンソルの積にも使用可能ですが、テンソルは末尾の（2階）行列のリストとして解釈され、
  
  その各々について行列積を取るという実装になっています。
<br />
<br />
- dot  
  matmulと同様の機能を提供する関数としてdotもよく使用されます。  

  行列やベクトルしか関係しない計算の場合、matmulとdotに差はありません。
  
  matmulとdotの差は3階以上のテンソルが絡む計算において現れることになりますが、ここでは省略します。（理由は後述）
<br />
<br />
- tensordot  
  テンソル積には積和を取る軸を指定できるtensordotを使用します（`C = np.tensordot(A, B, axes = ?)`）
  
  例えばA,Bがそれぞれ3階のテンソルの時は
  
  `axes=0`：$c_{ijklmn} = a_{ijk}b_{lmn}$
  
  `axes=1`：$c_{ijlm} = \sum_{k} a_{ijk}b_{klm}$
  
  `axes=2 (default)`：$c_{il} = \sum_{j,k} a_{ijk}b_{jkl} \left(\neq \sum_{j,k} a_{ijk}b_{kjl}\right)$
  
  ※ 他にもより直接的に軸を指定する方法もありますがここでは省略します
<br />
<br />
- einsum  
  テンソル積一般を表現でき、表現力の意味では最も強力です。（byアインシュタインの縮約規則）
  
  このeinsumがあればtraceやtransposeはもちろん、これまでに扱ったmatmulやtensordotも実現可能できてしまいます。（※要使い分け）
  
  重要なのは、einsumでは2テンソル間の演算だけでなく3つ以上のテンソル間の演算も一度に実現可能であることでしょう。
  
  なお、積和の式が与えられた時のeinsumの表現は一般に次のようにして得ることができます（例：$c_{ik} = \sum_{j} a_{ij}\times b_{jk}$）
  1. 変数名を消す（例：$ik = \sum_{j} ij \times jk$）
  2. 積（$\times$）をカンマで置き換える（例：$ik = \sum_{j} ij, jk$）
  3. シグマを消す（例：$ik = ij, jk$）
  4. 左辺右辺を反転させ、等号を->にする（例：$ij, jk -> ik$）
  
  （例）
  
  `np.einsum('ij,jk->ik', A, B)`：行列積$\sum_{j} a_{ij}b_{jk}$
  
  `np.einsum('i,i->', v, u)`：ベクトルの内積$\sum_{i} v_{i}u_{i}$

参考：
- https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html
- https://docs.scipy.org/doc/numpy/reference/generated/numpy.tensordot.html
- https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html

In [85]:
# matmul
np.matmul(mat2, converted1)

<IPython.core.display.Javascript object>

array([48863378.75567901, 300297600.9939416], dtype=object)

In [86]:
# dot
np.dot(mat2, converted1)

<IPython.core.display.Javascript object>

array([48863378.75567901, 300297600.9939416], dtype=object)

In [76]:
# tensordot
A = np.arange(18).reshape(2,3,3)
B = np.arange(36).reshape(3,3,4)

print(np.tensordot(A, B).shape)
print(np.tensordot(A, B, axes=1).shape)
print(np.tensordot(A, B, axes=0).shape)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(2, 4)


<IPython.core.display.Javascript object>

(2, 3, 3, 4)


<IPython.core.display.Javascript object>

(2, 3, 3, 3, 3, 4)


In [75]:
# einsum
A = np.arange(25).reshape(5,5)
b = np.arange(5)
b2 = np.arange(5) + 1

# トレース
print(np.einsum('ii', A))   # C = \sum_{i} a_{ii}
print(np.trace(A))
print()

# 転置
print(np.einsum('ji', A))   # c_{ij} = a_{ji}
print(A.T)
print()

# 内積
print(np.einsum('i,i', b, b2))  # C = \sum_{i} b_{i}*b2_{i}
print(np.inner(b, b2))
print()

# 対角成分
print(np.einsum('ii->i', A))    # c_{i} = a_{ii}
print(np.diag(A))
print()

# 軸指定和
print(np.einsum('ij->j', A))    # c_{j} = \sum_{i} a_{ij}
print(np.sum(A, axis=0))
print()

# 行列ベクトル積
print(np.einsum('ij,j->i', A, b))   # c_{i} = \sum_{j} a_{ij}*b_{j}
print(np.matmul(A, b))
print()

# outer product（いわゆる直積で、クロス積ではない）
print(np.einsum('i,j->ij', b, b2))  # c_{ij} = b_{i}*b2_{j}
print(np.outer(b, b2))
print()

# 行列積（matmul）
A = np.arange(6).reshape((3,2))
B = np.arange(12).reshape((4,3))
print(np.einsum('ij,ki->jk', A, B))     # c_{jk} = \sum_{i} a_{ij}*b_{ki}
print(np.matmul(A.T, B.T))
print()

# テンソル積（tensordot）
A = np.arange(18).reshape(2,3,3)
B = np.arange(36).reshape(3,3,4)
print(np.einsum('ijk,jkl->il', A, B))   # c_{il} = \sum_{j,k} a_{ijk}*b_{jkl}
print(np.tensordot(A, B))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

60


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[[ 816  852  888  924]
 [2112 2229 2346 2463]]


<IPython.core.display.Javascript object>

[[ 816  852  888  924]
 [2112 2229 2346 2463]]


#### !!! 注意 !!!（einsumとnewaxisや他の関数の使い分け）
einsumは先程の通り柔軟な表現力を持つ大変強力な関数ですから、

「einsumさえあればnewaxisによるBroadcastingやmatmulなど他の関数は不要だ」

と感じる人もいるかもしれません。ですが、

- 既に行列やベクトルの演算に対してはnewaxisを用いる方法やmatmul等の特定の演算用の関数が普及している

- 通常の行列やベクトルの演算にeinsumで対応するのは大げさで、コードも長くなりがち

等の理由からeinsumの濫用は望ましくありません。ただやはり、

- テンソルの演算に限れば、einsumとそれ以外の普及度の差が小さい

- テンソルの演算では大概演算が複雑で、newaxisやmatmul等の関数を使用すると演算内容が分かりずらく、コードの健全性を損ねる

というのも事実です。そこで本講座では

- 演算結果が行列やベクトル＝newaxisやmatmul等の関数

- 演算結果が(3階以上の)テンソル＝einsum

という使い分けの基準に沿って進めていきます。

(matmulとdotは3階以上のテンソルが出現する場合に違いが現れますが、こうした場合einsumを使用するべきでしょう)

### Broadcasting

#### - 概要

次元（や次元の長さ）の違う行列やベクトル間で、計算が成立するように"うまく"次元（や次元の長さ）を拡張してくれる機能。

例えば、`W.shape = (M,N)`, `x.shape = (1,N)`の場合、`W+x` は本来そのままでは計算できません。(`∵ W.shape ≠ x.shape`)

numpyでは暗黙に`x`について (1,N) => (M, N) と `x` をM回複製する事で `x` を `W` の`shape`に合わせてくれます。

※ 通常のN要素配列の`shape`は(N,)ですが、Broadcastingに際しては行ベクトル(1,N)として解釈されます（numpyのみ）

参考：
- https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html
- http://sucrose.hatenablog.com/entry/2014/12/15/000352

In [77]:
W = np.array([[1, 2, 3], [4, 5, 6]])
x = np.array([7, 8, 9])

print(np.ones((2,3))*x)
print(x*np.ones((2,3)))
print()

print(W * x)
print()

print(np.zeros((2,3))+x)
print(x+np.zeros((2,3)))
print()

print(W + x)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[[7. 8. 9.]
 [7. 8. 9.]]


<IPython.core.display.Javascript object>

[[7. 8. 9.]
 [7. 8. 9.]]

[[ 7 16 27]
 [28 40 54]]



<IPython.core.display.Javascript object>

[[7. 8. 9.]
 [7. 8. 9.]]


<IPython.core.display.Javascript object>

[[7. 8. 9.]
 [7. 8. 9.]]

[[ 8 10 12]
 [11 13 15]]


#### - numpy.newaxis

numpy.newaxisは配列や行列に対して新しい次元を追加できるオブジェクト。

例えば、N次元配列 `x` を考えると `x.shape = (N,)` ですが、numpy.newaxisを用いることで、

行ベクトル `x[numpy.newaxis, :].shape = (1,N)` としたり、

列ベクトル `x[:, numpy.newaxis].shape = (N,1)` としたりすることが可能です。

In [79]:
W = np.array([[1, 2, 3], [4, 5, 6]])
x = np.array([7, 8, 9])

print(W.shape)
print(W.T.shape)
print()

print(x.shape)
print(x.T.shape)  # x と x.T の shape は同じ
print()

print(x[np.newaxis, :])  # two dimension
print(x[np.newaxis, :].shape)
print(x[np.newaxis, :].T.shape)
print()

print(x[:, np.newaxis])  # two dimension with different shape
print(x[:, np.newaxis].shape)
print(x[:, np.newaxis].T.shape)
print()

print(x[np.newaxis].shape)  # x[np.newaxis,:]と同じ

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(2, 3)
(3, 2)

(3,)
(3,)



<IPython.core.display.Javascript object>

[[7 8 9]]


...

(3, 1)


<IPython.core.display.Javascript object>

(1, 3)



<IPython.core.display.Javascript object>

(1, 3)


In [ ]:
# numpy.newaxisを使ったbroadcasting
x = np.array([7, 8, 9])

print(np.ones((3,3))*x)                 # x:(3,)->(1,3)->(3,3)
print(np.ones((3,3))*x[np.newaxis, :])  # x:(1,3)->(3,3)
print(np.ones((3,3))*x[:, np.newaxis])  # x:(3,1)->(3,3)
print()

A = np.arange(9).reshape(3,3)
print(A)
print(A*x)
print(A*x[np.newaxis, :])
print(A*x[:, np.newaxis])

※ 先述の通り、Broadcastingに際してN要素の配列は<u>numpyでは</u>(1,N)として解釈されます。

　 しかし、本来N要素の配列は(N,1)と(1,N)のどちらかは区別がつくものではなく、<u>実際次週以降に扱うTensorFlowでは明示的にする必要があります。</u>

#### - Broadcastingの一般論

ここまで2次元までの単純な例を取り上げました。そろそろ慣れてきたところでBroadcastingの一般的なルールを次に示しましょう。

1. 各入力配列の次元数を最大の次元数に揃える。具体的には、足りない次元数だけ大きさ1の次元を"頭に"追加することで行う。
    
    （例）(2,3) + (3,) ⇒ (2,3) + (1,3)
    
    　　　(2,3,4,5) + (4,5) ⇒ (2,3,4,5) + (1,1,4,5)
<br />
<br />
2. 出力配列の各次元の大きさを入力配列の次元の大きさの最大値により決める。
    
    （例）(3,1) + (3,) ⇒ (<u>3</u>,1) + (1,<u>3</u>)  ∴出力次元は(3,3)
    
    　　　(2,3,1,5) + (4,5) ⇒ (<u>2</u>,<u>3</u>,1,<u>5</u>) + (1,1,<u>4</u>,<u>5</u>)  ∴出力次元は(2,3,4,5)
<br />
<br />
3. 全ての入力配列について、各次元の大きさが1であるか、対応する出力配列の次元の大きさに一致すればBroadcasting可能。
    
    （例）(3,1) + (3,) ⇒ (<u>3</u>,1) + (1,<u>3</u>)  ∴出力次元は(3,3)
    
    　　　　(3,1)は0次元目は3で出力配列に一致、1次元目は1なので問題ない。(3,)=(1,3)も同様に問題ないのでBroadcasting可能。
    
    　　　(2,3,1,3) + (4,5) ⇒ (<u>2</u>,<u>3</u>,1,3) + (1,1,<u>4</u>,<u>5</u>)  ∴出力次元は(2,3,4,5)
       
    　　　　(2,3,1,3)は3次元目が3(≠1)であるが出力配列の3次元目は5なのでBroadcasting不可能。
<br />
<br />
4. 各入力配列について、大きさが1である次元に関して複製を行い、出力次元に合わせる。

参考：
- http://docs.scipy.org/doc/numpy/reference/ufuncs.html#broadcasting

In [80]:
W = np.array([[1, 2, 3], [4, 5, 6]])
x = np.array([7, 8, 9])

print(np.ones((2,3))*x)                 # x:(3,)->(1,3)->(2,3)
print(np.ones((2,3))*x[np.newaxis, :])  # x:(1,3)->(2,3)
#print(np.ones((2,3))*x[:, np.newaxis]) # x:(3,1)->(2,3) unable
#print(np.ones((3,2))*x)                # x:(3,)->(1,3)->(3,2) unable
#print(np.ones((3,2))*x[np.newaxis, :]) # x:(1,3)->(3,2) unable
print(np.ones((3,2))*x[:, np.newaxis])  # x:(3,1)->(3,2)
print()

print(np.ones((3,3)) * x)
print(x[:, np.newaxis] * np.ones((3,3)))
print(x[:, np.newaxis] * x)
print()

print(W.T + x[np.newaxis, :].T)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[[7. 8. 9.]
 [7. 8. 9.]]


<IPython.core.display.Javascript object>

...

[[7. 7. 7.]
 [8. 8. 8.]
 [9. 9. 9.]]


<IPython.core.display.Javascript object>

[[49 56 63]
 [56 64 72]
 [63 72 81]]



<IPython.core.display.Javascript object>

[[ 8 11]
 [10 13]
 [12 15]]


# Reading real dataset

In [31]:
p = Path("../datasets/HONSHA-CHOUBO")
paths = list(p.glob("**/*.xls"))
xls1 = pd.read_excel(paths[0])
xls1.index

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

RangeIndex(start=0, stop=41, step=1)

In [32]:
xls1.columns

Index([ 'Unnamed: 0',          '備前',            10,          2011,
                '計画',  'Unnamed: 5',          '実績',  'Unnamed: 7',
               '計画差',         '前年差', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17'],
      dtype='object')

In [36]:
xls1.shape

(41, 18)

In [43]:
xls1.実績

0          数量
1     3831.85
2     24634.3
3     28466.1
4     14581.6
5         516
6           0
7           0
8        47.9
9           0
10          7
11       54.9
12          0
13          7
14          3
15          0
16          1
17          0
18          7
19          0
20          0
21          3
22          0
23      27181
24     104981
25         36
26          0
27          0
28        NaN
29        NaN
30       1267
31         63
32          1
33     303000
34        381
35        NaN
36        NaN
37        NaN
38        NaN
39        NaN
40        NaN
Name: 実績, dtype: object

In [58]:
count = 1
proccolumns = []
for column in xls1.columns:
    
    try:
        column:str = column
        if column.startswith("Unnamed"):
            proccolumns.append("column" + str(count))
        else:
            proccolumns.append(column)
              
        count+=1
    except:
        proccolumns.append(str(column))
        continue
proccolumns

['column1',
 '備前',
 '10',
 '2011',
 '計画',
 'column4',
 '実績',
 'column6',
 '計画差',
 '前年差',
 'column9',
 'column10',
 'column11',
 'column12',
 'column13',
 'column14',
 'column15',
 'column16']

In [59]:
xls1.columns = proccolumns
xls1

,column1,備前,10,2011,計画,column4,実績,column6,計画差,前年差,column9,column10,column11,column12,column13,column14,column15,column16
0,NaN,品名,利益＠,前年実績,数量,利益金額,数量,利益金額,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,燃料油,スーパーゼアス,11,4309.08,4500,49500,3831.85,42150.3,-668.150000,-477.230000,NaN,2011,現金,プリカ,現金＋プリカ,クレジット,掛け,累計
2,NaN,ゼアス,11,24379.4,25000,275000,24634.3,270977,-365.700000,254.870000,NaN,揮計,3004.67,1479.68,4484.35,11784.5,12419.7,28688.5
3,NaN,揮発油計,11,28688.5,29500,324500,28466.1,313128,-1033.850000,-222.360000,NaN,軽油,223.99,58.45,282.44,1988.87,11123.9,13395.2
4,NaN,軽 油,11,13395.2,14000,154000,14581.6,160398,581.630000,1186.440000,NaN,灯油,200,126,326,507,0,833
5,NaN,灯 油,10,833,1000,10000,516,5160,-484.000000,-317.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,産業用灯油,3,0,0,0,0,0,0.000000,0.000000,NaN,2012,現金,プリカ,現金＋プリカ,クレジット,掛け,累計
7,NaN,重油,12.3,0,0,0,0,0,0.000000,0.000000,NaN,揮計,3005.35,1882.55,4887.9,12040.5,11537.7,28466.1
8,ゼプロ,ゼプロ,450,94.5,82,36900,47.9,21555,-34.100000,-46.600000,NaN,軽油,165.25,20,185.25,2527.9,11868.5,14581.6
9,NaN,ＡＴＦ数量,600,0,18,10800,0,0,-18.000000,0.000000,NaN,灯油,114,210,324,96,96,516


# Scaling up analysis using database

In [1]:
import pyodbc 
[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

[]

In [ ]:
con_str = 'Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};Dbq={0};'.format(r"../datasets/mdb2000/Manage.mdb")
conn = pyodbc.connect(con_str)
cur = conn.cursor()
sql_create_table = ('select * from 実積;')
cur.execute(sql_create_table)
for row in cur.fetchall():
    print(row)

# Copyrights

All rights reserved for 吉延石油株式会社 and the publishers of scraped resources.

参考:

* https://weblab.t.u-tokyo.ac.jp/deep-learning基礎講座演習コンテンツ-公開ページ/